<a href="https://colab.research.google.com/github/JalesBussinguer/Remote_Sensing_Studies/blob/master/Desafio_Worcap_2020_90%2C13%25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação das bibliotecas básicas

import pandas as pd
import numpy as np
import io
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# Importação das bibliotecas e módulos de machine learning

from sklearn import ensemble, preprocessing, tree
from sklearn.metrics import auc, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV, KFold
from yellowbrick.classifier import ConfusionMatrix, ROCAUC
from yellowbrick.model_selection import learning_curve

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# Importando os dados de treino

df = "https://raw.githubusercontent.com/JalesBussinguer/Remote_Sensing_Studies/master/data/worcap_2020/dados_verdade_terrestre.csv"

dados = pd.read_csv(df)

dados.head()

,id,b1,b2,b3,b4,b5,b6,b7,b8,b9,pred_minus_obs_H_b1,pred_minus_obs_H_b2,pred_minus_obs_H_b3,pred_minus_obs_H_b4,pred_minus_obs_H_b5,pred_minus_obs_H_b6,pred_minus_obs_H_b7,pred_minus_obs_H_b8,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9,label
0,454,54,28,51,93,56,100,80,24,55,64.88,26.03,50.29,6.74,-30.83,-39.83,0.47,5.50,0.47,-24.52,-0.83,-3.99,-23.22,-0.94,-4.30,-23.22,-1.69,-4.55,s
1,457,54,29,51,88,51,91,95,24,55,56.28,20.62,42.67,10.16,-26.12,-32.05,-21.37,5.39,-0.30,-19.03,-0.54,-4.09,-22.24,-1.01,-4.93,-17.81,-0.53,-2.98,s
2,261,59,30,52,90,54,93,80,26,58,56.58,21.37,45.45,12.77,-28.17,-30.94,-1.30,3.64,-2.27,-21.04,-1.86,-5.61,-27.73,-1.15,-5.74,-21.81,-2.39,-5.07,s
3,8,53,27,49,95,49,92,63,25,54,66.97,24.43,49.28,8.08,-22.53,-28.25,19.78,3.75,0.92,-25.65,-2.09,-5.95,-39.27,-2.13,-8.73,-30.73,-2.42,-5.58,s
4,478,58,40,65,100,58,100,106,26,57,51.79,8.74,28.06,-1.00,-33.12,-41.04,-32.17,3.48,-2.08,-18.31,-0.20,-4.85,-21.15,-1.00,-4.84,-17.00,-0.91,-3.38,d


O conjunto de dados **`treino.csv`** será utilizado para construir os modelos de aprendizagem de máquina. Este apresenta a "verdade em terra" para cada classe por meio da coluna `label`.

In [ ]:
# Importação dos dados de validação

dados_validacao = pd.read_csv("https://raw.githubusercontent.com/JalesBussinguer/Remote_Sensing_Studies/master/data/worcap_2020/dados_validacao.csv")
dados_validacao.head(5)

dados_val_sid = dados_validacao.drop(columns="id")

O conjunto **`teste.csv`** será utilizado para verificar a capacidade de generalização do modelo criado.

## Dicionário de variáveis:

* `id`: Identificação única da linha, alusiva a um pixel único das imagens;
* `b1`: Banda do verde corresponde ao mês de setembro de 2010;
* `b2`: Banda do vermelho corresponde ao mês de setembro de 2010;
* `b3`: Banda do infravermelho próximo corresponde ao mês de setembro de 2010;
* `b4`: Banda do verde corresponde ao mês de Março de 2011;
* `b5`: Banda do vermelho corresponde ao mês de Março de 2011;
* `b6`: Banda do infravermelho próximo corresponde ao mês de Março de 2011;
* `b7`: Banda do verde corresponde ao mês de Maio de 2011;
* `b8`: Banda do vermelho corresponde ao mês de Maio de 2011;
* `b9`: Banda do infravermelho próximo corresponde ao mês de Maio de 2011;
* `pred_minus_obs_S_b1` até `pred_minus_obs_S_b9`: Valores espectrais previstos (Com base na interpolação espacial) subtraídos dos valores espectrais reais para a classe "s";
* `pred_minus_obs_H_b1` até `pred_minus_obs_H_b9`: Valores espectrais previstos (Com base na interpolação espacial) subtraídos dos valores espectrais reais para a classe "h".

## Rótulos:

* `S` = *Sugi* (Floresta composta predominantemente pela espécie [*Cryptomeria japonica*](https://pt.wikipedia.org/wiki/Cryptomeria_japonica))
* `H` = *Hinoki* (Floresta composta predominantemente pela espécie [*Chamaecyparis obtusa*](https://en.wikipedia.org/wiki/Chamaecyparis_obtusa))
* `D` = *Mixed Deciduous* (Formação florestal composta por diversas espécies deciduais ou caducifólias);
* `O` = *Other* (Feição não-florestal)

Por se tratar de um problema de classificação de múltiplas classes, os resultados obtidos com a utlização de Regressão Logística e KNN podem não ser acurados. Dessa forma, serão utilizados somente os algoritmos de Árvore de Decisão e Random Forest Classification.

# Análise Exploratória

Nesta primeira fase, é interessante verificar algumas características do dataset, para certificar que tudo está ok para a implementação do modelo de aprendizado de máquina.

## 1. Conferindo se existem dados faltando

In [ ]:
# Expressão que conta os dados que faltam pelo tamanho do dataset, retornando a porcentagem faltante de dados

dados.isnull().sum()/dados.shape[0]

id                     0.0
b1                     0.0
b2                     0.0
b3                     0.0
b4                     0.0
b5                     0.0
b6                     0.0
b7                     0.0
b8                     0.0
b9                     0.0
pred_minus_obs_H_b1    0.0
pred_minus_obs_H_b2    0.0
pred_minus_obs_H_b3    0.0
pred_minus_obs_H_b4    0.0
pred_minus_obs_H_b5    0.0
pred_minus_obs_H_b6    0.0
pred_minus_obs_H_b7    0.0
pred_minus_obs_H_b8    0.0
pred_minus_obs_H_b9    0.0
pred_minus_obs_S_b1    0.0
pred_minus_obs_S_b2    0.0
pred_minus_obs_S_b3    0.0
pred_minus_obs_S_b4    0.0
pred_minus_obs_S_b5    0.0
pred_minus_obs_S_b6    0.0
pred_minus_obs_S_b7    0.0
pred_minus_obs_S_b8    0.0
pred_minus_obs_S_b9    0.0
label                  0.0
dtype: float64

No conjunto de dados de treino, não há dados faltantes. Portanto, podemos presseguir na análise.

In [ ]:
dados_validacao.isnull().sum()/dados_validacao.shape[0]

id                     0.0
b1                     0.0
b2                     0.0
b3                     0.0
b4                     0.0
b5                     0.0
b6                     0.0
b7                     0.0
b8                     0.0
b9                     0.0
pred_minus_obs_H_b1    0.0
pred_minus_obs_H_b2    0.0
pred_minus_obs_H_b3    0.0
pred_minus_obs_H_b4    0.0
pred_minus_obs_H_b5    0.0
pred_minus_obs_H_b6    0.0
pred_minus_obs_H_b7    0.0
pred_minus_obs_H_b8    0.0
pred_minus_obs_H_b9    0.0
pred_minus_obs_S_b1    0.0
pred_minus_obs_S_b2    0.0
pred_minus_obs_S_b3    0.0
pred_minus_obs_S_b4    0.0
pred_minus_obs_S_b5    0.0
pred_minus_obs_S_b6    0.0
pred_minus_obs_S_b7    0.0
pred_minus_obs_S_b8    0.0
pred_minus_obs_S_b9    0.0
dtype: float64

No conjunto de dados de teste também não existem dados faltantes. Portanto, prosseguimos para uma análise dos tipos de dados que temos.

In [ ]:
# Verificação dos tipos de dados no conjunto de dados de treino

dados.dtypes

id                       int64
b1                       int64
b2                       int64
b3                       int64
b4                       int64
b5                       int64
b6                       int64
b7                       int64
b8                       int64
b9                       int64
pred_minus_obs_H_b1    float64
pred_minus_obs_H_b2    float64
pred_minus_obs_H_b3    float64
pred_minus_obs_H_b4    float64
pred_minus_obs_H_b5    float64
pred_minus_obs_H_b6    float64
pred_minus_obs_H_b7    float64
pred_minus_obs_H_b8    float64
pred_minus_obs_H_b9    float64
pred_minus_obs_S_b1    float64
pred_minus_obs_S_b2    float64
pred_minus_obs_S_b3    float64
pred_minus_obs_S_b4    float64
pred_minus_obs_S_b5    float64
pred_minus_obs_S_b6    float64
pred_minus_obs_S_b7    float64
pred_minus_obs_S_b8    float64
pred_minus_obs_S_b9    float64
label                   object
dtype: object

Observa-se que no conjunto de dados de treino, existem três tipos de dados distintos:
* Inteiros (int64): são os dados que representam os números digitais expressos nos pixels das imagens, e são diretamente relacionados à resposta espectral dos alvos da cena. Na coluna `id`, representam a identificação do pixel.
* Flutuantes (float64): são dados que representam o resultado de uma estatística que mede a diferença entre uma resposta espectral modelada e a resposta espectral observada;
* Texto (object): são os dados de rótulos dos pixels, representando a verdade terrestre das cenas.

In [ ]:
# Verificação dos tipos de dados no conjunto de dados de treino

dados_validacao.dtypes

id                       int64
b1                       int64
b2                       int64
b3                       int64
b4                       int64
b5                       int64
b6                       int64
b7                       int64
b8                       int64
b9                       int64
pred_minus_obs_H_b1    float64
pred_minus_obs_H_b2    float64
pred_minus_obs_H_b3    float64
pred_minus_obs_H_b4    float64
pred_minus_obs_H_b5    float64
pred_minus_obs_H_b6    float64
pred_minus_obs_H_b7    float64
pred_minus_obs_H_b8    float64
pred_minus_obs_H_b9    float64
pred_minus_obs_S_b1    float64
pred_minus_obs_S_b2    float64
pred_minus_obs_S_b3    float64
pred_minus_obs_S_b4    float64
pred_minus_obs_S_b5    float64
pred_minus_obs_S_b6    float64
pred_minus_obs_S_b7    float64
pred_minus_obs_S_b8    float64
pred_minus_obs_S_b9    float64
dtype: object

O conjunto de dados de teste apresenta a mesma estrutura de tipos de dados que o conjunto de treino. Uma vez que os conjuntos de dados são compatíveis estruturalmente, podemos seguir para a verificação de valores únicos.

In [ ]:
# Valores únicos (dados de treino)

dados.nunique()

id                     209
b1                      48
b2                      52
b3                      49
b4                      52
b5                      42
b6                      47
b7                      67
b8                      28
b9                      36
pred_minus_obs_H_b1    206
pred_minus_obs_H_b2    199
pred_minus_obs_H_b3    200
pred_minus_obs_H_b4    202
pred_minus_obs_H_b5    200
pred_minus_obs_H_b6    206
pred_minus_obs_H_b7    204
pred_minus_obs_H_b8    185
pred_minus_obs_H_b9    197
pred_minus_obs_S_b1    190
pred_minus_obs_S_b2    155
pred_minus_obs_S_b3    179
pred_minus_obs_S_b4    195
pred_minus_obs_S_b5    108
pred_minus_obs_S_b6    163
pred_minus_obs_S_b7    194
pred_minus_obs_S_b8    154
pred_minus_obs_S_b9    170
label                    4
dtype: int64

In [ ]:
# Valores únicos (dados de teste)

dados_validacao.nunique()

id                     314
b1                      58
b2                      55
b3                      54
b4                      59
b5                      49
b6                      44
b7                      72
b8                      39
b9                      46
pred_minus_obs_H_b1    300
pred_minus_obs_H_b2    296
pred_minus_obs_H_b3    301
pred_minus_obs_H_b4    300
pred_minus_obs_H_b5    291
pred_minus_obs_H_b6    294
pred_minus_obs_H_b7    304
pred_minus_obs_H_b8    269
pred_minus_obs_H_b9    284
pred_minus_obs_S_b1    272
pred_minus_obs_S_b2    227
pred_minus_obs_S_b3    251
pred_minus_obs_S_b4    282
pred_minus_obs_S_b5    139
pred_minus_obs_S_b6    227
pred_minus_obs_S_b7    278
pred_minus_obs_S_b8    213
pred_minus_obs_S_b9    226
dtype: int64

Vamos verificar agora a estatística descritiva do conjunto de dados de treino.

In [ ]:
dados.describe()

,id,b1,b2,b3,b4,b5,b6,b7,b8,b9,pred_minus_obs_H_b1,pred_minus_obs_H_b2,pred_minus_obs_H_b3,pred_minus_obs_H_b4,pred_minus_obs_H_b5,pred_minus_obs_H_b6,pred_minus_obs_H_b7,pred_minus_obs_H_b8,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9
count,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000
mean,279.473684,60.028708,39.818182,62.535885,98.631579,58.062201,99.478469,88.521531,27.349282,59.143541,53.995167,11.101866,33.790718,-0.387799,-32.808708,-39.359761,-11.722823,2.252727,-3.755311,-20.445407,-0.892392,-4.196411,-22.068086,-0.989187,-4.636411,-19.295455,-1.585502,-4.115407
std,157.575777,11.889278,18.148041,18.283029,12.353104,11.212157,10.326905,16.248987,7.189570,8.277545,12.278955,18.650286,19.208383,12.807750,11.161881,10.171291,16.608997,7.242365,8.320464,3.946111,1.425159,1.875246,4.871465,0.520297,1.314000,4.398123,1.355239,1.560729
min,2.000000,31.000000,23.000000,47.000000,75.000000,44.000000,83.000000,45.000000,19.000000,48.000000,7.660000,-112.600000,-106.120000,-51.160000,-74.560000,-77.170000,-56.980000,-54.740000,-58.280000,-32.240000,-6.020000,-9.730000,-40.370000,-2.550000,-8.730000,-34.140000,-7.990000,-10.810000
25%,140.000000,52.000000,28.000000,51.000000,90.000000,50.000000,92.000000,77.000000,24.000000,54.000000,45.970000,4.040000,28.190000,-7.580000,-38.330000,-44.980000,-23.480000,2.860000,-4.540000,-23.070000,-1.740000,-5.330000,-24.760000,-1.190000,-5.500000,-22.500000,-2.190000,-4.830000
50%,278.000000,58.000000,31.000000,55.000000,96.000000,54.000000,97.000000,89.000000,25.000000,57.000000,55.130000,19.000000,40.190000,2.010000,-28.810000,-36.300000,-11.260000,4.210000,-1.990000,-20.220000,-0.830000,-4.350000,-21.640000,-0.980000,-4.650000,-19.200000,-1.480000,-4.030000
75%,428.000000,67.000000,48.000000,67.000000,107.000000,64.000000,106.000000,101.000000,27.000000,60.000000,62.310000,22.880000,45.490000,8.680000,-24.780000,-32.140000,0.110000,5.530000,0.950000,-18.390000,-0.220000,-2.880000,-19.030000,-0.770000,-3.870000,-16.800000,-0.950000,-3.240000
max,522.000000,105.000000,160.000000,196.000000,152.000000,100.000000,138.000000,129.000000,84.000000,114.000000,86.080000,29.900000,52.710000,21.780000,-19.490000,-23.550000,34.500000,10.830000,8.280000,5.130000,12.460000,7.370000,1.880000,3.440000,3.940000,3.670000,8.840000,7.790000


# Construção do modelo:

## 1.  Pré-processamento:

In [ ]:
# Remoção da coluna de identificação do conjunto de dados de treino

dados = dados.drop(columns="id")

In [ ]:
# Separação dos atributos e dos rótulos

X = dados.iloc[:,:-1].values # atributos (amostra de pixels)
y = dados.iloc[:,-1].values # rótulos (verdade terreste)

In [ ]:
# Separação das amostras de treino e de teste (80% - treino e 20% - teste)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
# Usamos o random_state = 42 paraeliminar a aleatoriedade caso venhamos a comparar diferentes modelos

# One-hot encoding
encoder = preprocessing.OneHotEncoder(sparse=False)
y_treino = encoder.fit_transform(y_treino.reshape(-1, 1))
y_teste = encoder.fit_transform(y_teste.reshape(-1, 1))
# Informações importantes

print("Formato dos dados de treino:", "X_treino =", X_treino.shape, "e y_treino =", y_treino.shape)
print("Formato dos dados de teste:", "X_teste =", X_teste.shape, "e y_teste =", y_teste.shape)
print()
print("Número de classes:", len(set(dados['label'].values)))
print("Número de features:", X_treino.shape[1])

Formato dos dados de treino: X_treino = (167, 27) e y_treino = (167, 4)
Formato dos dados de teste: X_teste = (42, 27) e y_teste = (42, 4)

Número de classes: 4
Número de features: 27


Um indicativo de que tudo está ok é o fato do número de linhas ser igual para os conjuntos de dados de treino. O conjunto x possui 209 linhas (valores) e 27 colunas (features), enquanto o conjunto y possui 209 linhas (valores) e um única coluna com os rótulos.

Para o conjunto de teste, o indicativo de que está tudo certo é a quantidade de colunas (features), que bate com a quantidade de features do conjunto de treino.

# 3. Criando os modelos

## *Random Forest Classifier (default)*

In [ ]:
rf_default = ensemble.RandomForestClassifier(n_estimators=1000, criterion='entropy', n_jobs=-1)

In [ ]:
rf_default.fit(X_treino, y_treino)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [ ]:
rf_default.score(X_teste, y_teste)

0.8809523809523809

## Grid Search

In [ ]:
rfc = ensemble.RandomForestClassifier()

In [ ]:
## Random Search
## N Estimators
n_estimators = [25, 50, 75, 100, 250, 500, 750, 1000, 1500, 2500]
## Min Samples Split
min_samples_split = range(1,5)
## Min Samples Leaf
min_samples_leaf = range(1,5)
## Max Features
max_features = ['auto', None]
# Max depth
max_depth = range(1,50)
# Criterion
criterion = ['gini', 'entropy']
# Criando o Random Search
rf_parametros = {
                'n_estimators': n_estimators,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'max_depth': max_depth,
                'criterion': criterion,
                'max_features': max_features}
## Random Forest Tuning
RFtuning = GridSearchCV(rfc, ## Random Forest
                              param_grid=rf_parametros, ## Parâmetros
                              cv=KFold(n_splits=4, shuffle=True, random_state=42), ## Cross validation
                              scoring = 'accuracy', ## Métrica
                              n_jobs = -1, ## Utilizando todos os processadores
                              verbose = 1).fit(X_treino, y_treino)

print(f'Melhor resultado: {RFtuning.best_score_} para {RFtuning.best_params_}')

Fitting 4 folds for each of 31360 candidates, totalling 125440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

## Resultado final

In [ ]:
rf = ensemble.RandomForestClassifier(n_estimators=1000, min_samples_split=2, min_samples_leaf=7,
                                     max_features=None, criterion='entropy', n_jobs=-1)

In [ ]:
rf.fit(X_treino, y_treino)

In [ ]:
rf.score(X_teste, y_teste)

In [ ]:
y_verificacao = rf_default.predict(dados_val_sid)

In [ ]:
y_verificacao = encoder.inverse_transform(y_verificacao).reshape(-1)

In [ ]:
y_verificacao

In [ ]:
index = dados_validacao["id"]
columns = ['label']


In [ ]:
output = pd.DataFrame(y_verificacao, index=index, columns=columns)

In [ ]:
output.to_csv('result.csv', sep=",")

In [ ]:

pd.read_csv('result.csv')

In [ ]:
from google.colab import files
files.download("result.csv")

In [ ]:
A